In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    "D:/datasets/train",
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    "D:/datasets/test",
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 8751 images belonging to 38 classes.
Found 10547 images belonging to 38 classes.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')  # Output layer
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\indum\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=test_generator
)

test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc}')

C:\Users\indum\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
274/274 ━━━━━━━━━━━━━━━━━━━━ 319s 1s/step - accuracy: 0.2361 - loss: 3.0942 - val_accuracy: 0.4550 - val_loss: 1.9365
Epoch 2/5
274/274 ━━━━━━━━━━━━━━━━━━━━ 305s 1s/step - accuracy: 0.4762 - loss: 1.8431 - val_accuracy: 0.5136 - val_loss: 1.8325
Epoch 3/5
274/274 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.5880 - loss: 1.4177 - val_accuracy: 0.5567 - val_loss: 1.7379
Epoch 4/5
274/274 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.6321 - loss: 1.2089 - val_accuracy: 0.5631 - val_loss: 1.6831
Epoch 5/5
274/274 ━━━━━━━━━━━━━━━━━━━━ 1636s 6s/step - accuracy: 0.6837 - loss: 1.0607 - val_accuracy: 0.6379 - val_loss: 1.3214
330/330 ━━━━━━━━━━━━━━━━━━━━ 63s 191ms/step - accuracy: 0.6384 - loss: 1.3189
Test Accuracy: 0.6379064917564392


In [8]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np

In [10]:
root = tk.Tk()
root.title("Image Classifier")

''

In [12]:
def upload_image():
    file_path = filedialog.askopenfilename()
    img = Image.open(file_path).resize((150, 150))
    img_array = np.array(img) / 255.0  # Normalize image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    result = list(train_generator.class_indices.keys())[class_index]
    
    label_result.config(text=f"Prediction: {result}")
    
    img_tk = ImageTk.PhotoImage(img)
    label_img.config(image=img_tk)
    label_img.image = img_tk

In [ ]:
button_upload = tk.Button(root, text="Upload Image", command=upload_image)
button_upload.pack()

In [ ]:
 label_result = tk.Label(root, text="Prediction: ")
label_result.pack()


In [ ]:
label_img = tk.Label(root)
label_img.pack()

root.mainloop()